<a href="https://colab.research.google.com/github/matteraggi/FineTuningAI/blob/main/vsProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**INSTALLS**

In [ ]:
!pip install --user transformers torch datasets peft tf-keras accelerate bitsandbytes trl evaluate radon zss

!pip install datasets --upgrade --force-reinstall

!pip install evaluate transformers openai human-eval

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 59.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 33.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 905.2 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 63.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 32.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.3/52.3 kB 4.4 MB/s eta 0:00:00
ERROR: Operation cancelled by user


**IMPORTS**

In [ ]:
# Importing stock ml libraries
import numpy as np
import pandas as pd
from sklearn import metrics
import transformers
import torch
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
# from transformers import BertTokenizer, BertModel, BertConfig
from datasets import load_dataset
from trl import SFTTrainer
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments,TrainingArguments,pipeline, logging,BitsAndBytesConfig
from peft import LoraConfig, PeftModel

**VARIABLES**

In [ ]:
# The model that you want to train from the Hugging Face hub
model_name = "bigcode/starcoder2-3b"  # Standard.  Good choice for a powerful and widely used LLM.

# The instruction dataset to use
dataset_name = "bigcode/self-oss-instruct-sc2-exec-filter-50k"  # Standard. A popular dataset for instruction tuning.

# Fine-tuned model name
new_model = "starcoder2-finetuned"  # Standard.  Clear and descriptive.

################################################################################
# QLoRA parameters
################################################################################

# LoRA attention dimension. It determines the size and parameter count of the low-rank adaptation
lora_r = 16  # Medium-high.  This is a reasonable value.  You could experiment with lower values (e.g., 32) for even more memory efficiency or higher values (e.g., 128) for potentially better performance (but more memory usage).

# Alpha parameter for LoRA scaling factor that determines the impact of the low-rank matrices on the original model's output.
# Controls the overall strength of the low-rank adaptation.
lora_alpha = 2  # Standard.  Often set to 2-4 times `lora_r`.  This is a common setting.

# Dropout probability for LoRA layers
lora_dropout = 0.05  # Standard.  A common dropout rate.  You could try slightly lower values (e.g., 0.05) if you're not seeing overfitting.

################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
use_4bit = True  # Standard. Essential for fitting large models on consumer GPUs.

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"  # Standard.  Using float16 can speed up computation if your GPU supports it.  If you encounter issues, try "bfloat16" (if your GPU supports it) or "float32".

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"  # Standard.  nf4 (NormalFloat4) is generally preferred.

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False  # Standard.  Double quantization can sometimes improve performance but increases complexity.  It's often left disabled initially.

################################################################################
# TrainingArguments parameters
################################################################################

# Output directory where the model predictions and checkpoints will be stored
output_dir = "./results"  # Standard.

# Number of training epochs
num_train_epochs = 1  # Low.  One epoch is very low.  You'll almost certainly need more epochs (e.g., 3-5 or even more) to get good results.

# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = True  # Standard.  You can enable this if your GPU supports it and `bnb_4bit_compute_dtype` is also set to a compatible value (float16).
bf16 = False  # Standard.  bf16 is generally preferred over fp16 if your GPU supports it.  Not available on a T4.

# Batch size per GPU for training
per_device_train_batch_size = 1  # Low.  You could try increasing this to 8 or even 16 if your GPU memory allows.  Higher batch sizes can often lead to faster training.

# Batch size per GPU for evaluation
per_device_eval_batch_size = 1  # Low.  Same as training batch size.

# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 4  # Standard.  If you run out of memory, you can increase this to 2, 4, or higher.  This effectively increases the batch size.

# Enable gradient checkpointing
gradient_checkpointing = False  # CHANGED DA FRIGO DA PROBLEMI RISOLVERE Standard.  Essential for reducing memory usage, especially with larger models.

# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3  # Standard.  A common value for gradient clipping.

# Initial learning rate (AdamW optimizer)
learning_rate = 2e-4  # Standard.  A good starting point.  You might want to experiment with values between 1e-4 and 5e-4.

# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.001  # Standard.

# Optimizer to use
optim = "paged_adamw_32bit"  # Standard.  A good optimizer.

# Learning rate schedule
lr_scheduler_type = "cosine"  # Standard.  A common and effective learning rate scheduler.

# Number of training steps (overrides num_train_epochs)
max_steps = 5000  # Standard.  Using `num_train_epochs` is usually sufficient.

# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03  # Standard.

# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True  # Standard.  Highly recommended for efficiency.

# Save checkpoint every X updates steps
save_steps = 500  # Low.  You should set this to a value greater than 0 (e.g., every 500 or 1000 steps) to save checkpoints during training.

# Log every X updates steps
logging_steps = 25  # Standard.

################################################################################
# SFT parameters
################################################################################

# Maximum sequence length to use
max_seq_length = 1024  # Important.  You *must* set this.  A common value is 2048 or 4096, but it depends on your data and GPU memory.  Start with a lower value and increase it if possible.

# Pack multiple short examples in the same input sequence to increase efficiency
packing = False  # Standard.  Can improve efficiency if your dataset has many short sequences.

# Load the entire model on the GPU 0
device_map = {"": 0}  # Standard.  This maps the model to the first GPU.

**DATASET AND INIT OF MODELS**

In [ ]:
torch.cuda.empty_cache()

# Load dataset (you can process it here)
dataset = load_dataset(dataset_name, split="train")

# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)


model.config.use_cache = False
model.config.pretraining_tp = 1
#if gradient_checkpointing:
#  model.gradient_checkpointing_enable()

# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = 'right'

# Load LoRA configuration
# Example configuration for target modules (can vary per model)
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "v_proj"]  # These are common modules for transformers (queries and values in attention layers)
)


# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard",
    gradient_checkpointing=gradient_checkpointing,
)

# Inspect dataset columns
print(dataset.column_names)



RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx

**CHANGES HEADERS OF DATASET**

In [ ]:
def preprocess_function(examples):
    # Concatenate 'instruction' and 'prompt' for each example in the batch
    input_texts = []
    for instruction, prompt in zip(examples['instruction'], examples['prompt']):
        if instruction:  # If 'instruction' exists
            input_texts.append(instruction + " " + prompt)
        else:
            input_texts.append(prompt)

    # Tokenize the input texts
    return tokenizer(input_texts, padding='max_length', truncation=True, max_length=128)

# Apply tokenization
tokenized_dataset = dataset.map(preprocess_function, batched=True)



Map:   0%|          | 0/50661 [00:00<?, ? examples/s]

**TRAINING**

In [ ]:
# Apply LoRA using PEFT
from peft import get_peft_model

for param in model.parameters():
    if param.dtype in [torch.float32, torch.float64, torch.float16, torch.bfloat16]:  # Check for floating point types
        param.requires_grad = True  # Ensure only floating-point parameters require gradients

print(any(param.requires_grad for param in model.parameters() if param.dtype in [torch.float32, torch.float64, torch.float16, torch.bfloat16]))  # Should print True

# Wrap the model with the LoRA configuration
model = get_peft_model(model, peft_config)


# Now, only LoRA layers will have requires_grad=True
print(any(param.requires_grad for param in model.parameters()))  # This should be True for LoRA layers

# Train the model with the modified configuration
trainer = SFTTrainer(
    model=model,
    train_dataset=tokenized_dataset,
    peft_config=peft_config,
    args=training_arguments,
)

# Start training
trainer.train()

# Save the fine-tuned model
trainer.model.save_pretrained(new_model)

True
True


Step,Training Loss
25,3.234500
50,3.448800
75,3.365400
100,3.023800
125,2.826600
150,2.303600
175,1.960500
200,1.841500
225,1.556800
250,1.519000


**NON RUNNARE DI DEFAULT**
Salvare modello che hai appena trainato sul tuo Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
model.save_pretrained("/content/drive/MyDrive/starcoder2-finetuned")
tokenizer.save_pretrained("/content/drive/MyDrive/starcoder2-finetuned")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


('/content/drive/MyDrive/starcoder2-finetuned/tokenizer_config.json',
 '/content/drive/MyDrive/starcoder2-finetuned/special_tokens_map.json',
 '/content/drive/MyDrive/starcoder2-finetuned/vocab.json',
 '/content/drive/MyDrive/starcoder2-finetuned/merges.txt',
 '/content/drive/MyDrive/starcoder2-finetuned/added_tokens.json',
 '/content/drive/MyDrive/starcoder2-finetuned/tokenizer.json')

**NON RUNNARE DI DEFAULT**
Per pushare il modello che hai appena trainato su git

In [ ]:
from google.colab import userdata
!apt-get install git
!git config --global user.email {userdata.get('GitEmail')}
!git config --global user.name {userdata.get('GitUsername')}
!git clone https://github.com/matteraggi/FineTuningAI.git
#!cd FineTuningAI  # Go to the *existing* FineTuningAI directory
#!mv starcoder2-finetuned FineTuningAI/starcoder2_Model_Tensors/
!cd FineTuningAI && git add starcoder2_Model_Tensors
!cd FineTuningAI && git commit -m "Add fine-tuned model"
!git config --global credential.helper store  # Or 'cache' if you prefer
!cd FineTuningAI && git push https://{userdata.get('PAT')}@github.com/matteraggi/FineTuningAI.git main


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git is already the newest version (1:2.34.1-1ubuntu1.12).
0 upgraded, 0 newly installed, 0 to remove and 18 not upgraded.
On branch main
Your branch is ahead of 'origin/main' by 1 commit.
  (use "git push" to publish your local commits)

nothing to commit, working tree clean
Enumerating objects: 7, done.
Counting objects: 100% (7/7), done.
Delta compression using up to 2 threads
Compressing objects: 100% (6/6), done.
Writing objects: 100% (6/6), 16.10 MiB | 4.45 MiB/s, done.
Total 6 (delta 1), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (1/1), completed with 1 local object.
To https://github.com/matteraggi/FineTuningAI.git
   84473d3..c0a8b3e  main -> main


**NON RUNNARE DI DEFAULT**
Loada il vecchio modello trainato da te dal tuo drive

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer #NOT TESTED

# Load the fine-tuned model from Google Drive
model_path = "/content/drive/MyDrive/starcoder2-finetuned"  # Path to your saved model

device = "cuda" if torch.cuda.is_available() else "cpu" # Get device

quantization_config = BitsAndBytesConfig(
      load_in_4bit=True,
      bnb_4bit_quant_type="nf4",  # Or "fp4" if you used that during training
      bnb_4bit_compute_dtype=torch.float16,  # Adjust if needed
      )
model = AutoModelForCausalLM.from_pretrained(
            model_path,
            device_map={"": device},
            quantization_config=quantization_config,
        ).eval()



**NON RUNNARE DI DEFAULT**
Loada il vecchio modello dal git

In [ ]:
import os # NOT TESTED
!apt-get install git
!git clone https://github.com/matteraggi/FineTuningAI.git
# Percorso alla directory del modello
model_dir = "FineTuningAI/starcoder2_Model_Tensors"

# Verifica se la directory esiste
if os.path.exists(model_dir):
    print("Directory del modello trovata!")

    from transformers import AutoModelForCausalLM, AutoTokenizer
    device = "cuda" if torch.cuda.is_available() else "cpu" # Get device

    quantization_config = BitsAndBytesConfig(
      load_in_4bit=True,
      bnb_4bit_quant_type="nf4",  # Or "fp4" if you used that during training
      bnb_4bit_compute_dtype=torch.float16,  # Adjust if needed
      )
    model = AutoModelForCausalLM.from_pretrained(
            model_dir,
            device_map={"": device},
            quantization_config=quantization_config,
        ).eval()

else:
    print(f"Errore: Directory del modello non trovata: {model_dir}")

Cloning into 'FineTuningAI'...
remote: Enumerating objects: 80, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 80 (delta 4), reused 2 (delta 1), pack-reused 69 (from 1)
Receiving objects: 100% (80/80), 42.69 MiB | 15.71 MiB/s, done.
Resolving deltas: 100% (37/37), done.
Directory del modello trovata!


**Caricare il benchmark HumanEval**
Dopo il fine-tuning, devi confrontare il tuo modello con uno pre-addestrato (StarCoder2 senza fine-tuning) e con il modello fine-tunato.

In [ ]:
SYSTEM_PROMPT = "Generate Python code for the following task:\n"
NON_FINETUNED_ADDITIONAL_PROMPT = ""

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
import os
import gc
import bitsandbytes as bnb
logging.set_verbosity(logging.CRITICAL)

# Load HumanEval dataset
humaneval = load_dataset("openai/openai_humaneval", split="test")

quantization_config = BitsAndBytesConfig(
  load_in_4bit=True,
  bnb_4bit_quant_type="nf4",  # Or "fp4" if you used that during training
  bnb_4bit_compute_dtype=torch.float16,  # Adjust if needed
  )
device = "cuda" if torch.cuda.is_available() else "cpu" # Get device
# Load base model and tokenizer
base_model_name = "bigcode/starcoder2-3b"  # Base model
non_finetuned_model= AutoModelForCausalLM.from_pretrained(
            model_name,
            device_map={"": device},
            quantization_config=quantization_config,
        ).eval()
# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model_name)

model_path = "FineTuningAI/starcoder2_Model_Tensors"  # CAREFUL CAN CHANGE DEPENDING IF TESTING OLD OR NEW

def generate_code(current_model, prompt, is_finetuned=False):
    if not is_finetuned:
        prompt = NON_FINETUNED_ADDITIONAL_PROMPT + prompt  # Prepend system prompt for the base model

    prompt = SYSTEM_PROMPT + prompt #add system prompt

    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    with torch.no_grad():
        outputs = current_model.generate(**inputs,max_new_tokens=200)

    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Evaluate base model and fine-tuned model
base_results = []
fine_tuned_results = []

for i in range(0, len(humaneval), 1): # Process 1 example at a time
    example = humaneval[i]
    prompt = example['prompt']

    base_code = generate_code(non_finetuned_model, prompt, is_finetuned=False)
    base_results.append({"prompt": prompt, "code": base_code})

    fine_tuned_code = generate_code(model, prompt, is_finetuned=True)
    fine_tuned_results.append({"prompt": prompt, "code": fine_tuned_code})
    print("done number"+ str(i))
    gc.collect() # Force garbage collection

# Display some results
print("Base Model Results:", base_results[:3])
print("Fine-tuned Model Results:", fine_tuned_results[:3])


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for o

KeyboardInterrupt: 

**RECOMMENDED** SAVE IN JSON FOR SAVING

In [ ]:
import json

# Assuming base_results and fine_tuned_results are your arrays
results = {
    "base_results": base_results,
    "fine_tuned_results": fine_tuned_results
}

# Save as JSON
with open("model_results.json", "w") as f:
    json.dump(results, f, indent=4)



**SAVE IN DRIVE**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
# Save as JSON in Drive
with open("/content/drive/MyDrive/model_results.json", "w") as f:
    json.dump(results, f, indent=4)



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**SAVE IN GIT**

In [ ]:
from google.colab import userdata
!apt-get install git
!git config --global user.email {userdata.get('GitEmail')}
!git config --global user.name {userdata.get('GitUsername')}
!git clone https://github.com/matteraggi/FineTuningAI.git
!cp model_results.json FineTuningAI/results/
!cd FineTuningAI && git add results/model_results.json
!cd FineTuningAI && git commit -m "Save model results"
!git config --global credential.helper store  # Or 'cache' if you prefer
!cd FineTuningAI && git push https://{userdata.get('PAT')}@github.com/matteraggi/FineTuningAI.git main


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git is already the newest version (1:2.34.1-1ubuntu1.12).
0 upgraded, 0 newly installed, 0 to remove and 18 not upgraded.
fatal: destination path 'FineTuningAI' already exists and is not an empty directory.
On branch main
Your branch is ahead of 'origin/main' by 2 commits.
  (use "git push" to publish your local commits)

nothing to commit, working tree clean
Everything up-to-date


**LOAD FROM DRIVE**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import json

# Load JSON file from Drive
with open("/content/drive/MyDrive/model_results.json", "r") as f:
    results = json.load(f)

# Now results['base_results'] and results['fine_tuned_results'] hold the data
base_results = results['base_results']
fine_tuned_results = results['fine_tuned_results']


Mounted at /content/drive


**LOAD FROM GIT**

In [ ]:
import json
!apt-get install git
!git clone https://github.com/matteraggi/FineTuningAI.git

with open("FineTuningAI/results/model_results.json", "r") as f:
    results = json.load(f)

base_results = results['base_results']
fine_tuned_results = results['fine_tuned_results']

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git is already the newest version (1:2.34.1-1ubuntu1.12).
0 upgraded, 0 newly installed, 0 to remove and 18 not upgraded.
fatal: destination path 'FineTuningAI' already exists and is not an empty directory.


**PRINT THE RESULTS**

In [ ]:
import pandas as pd
from tabulate import tabulate

table_data = []
for non_fine_tuned, fine_tuned in zip(base_results, fine_tuned_results):
    table_data.append({
        "Prompt": non_fine_tuned['prompt'],
        "Non-Finetuned Code": non_fine_tuned['code'],
        "Finetuned Code": fine_tuned['code']
    })
# Use tabulate for better formatting
df = pd.DataFrame(table_data)

# Print only the first 5 rows with tabulate
print(tabulate(df.head(), headers='keys', tablefmt='grid', numalign="left", stralign="left"))

+----+--------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------+
|    | Prompt                                                                                                       | Non-Finetuned Code                                                                                                                                                                                                  | Finetuned Code                                                                                                       |
+====+============================================================================================

**eval with pass@k** see if any run

In [ ]:
import json
import pandas as pd
import evaluate
from typing import List
import re
import os
os.environ["HF_ALLOW_CODE_EVAL"] = "1"

# Load HumanEval dataset
code_eval_metric = evaluate.load("code_eval")

def evaluate_responses(responses, canonical_solution, k_values=([1])):
    """Evaluates LM responses using pass@k metric.

    Args:
        responses: A list of string responses (code generations).
        canonical_solution: The ground truth canonical solution.
        k_values: Tuple of k values to calculate pass@k.

    Returns:
        A dictionary containing pass@k results for each k. Returns an empty dict on error.
    """
    try:
        # Ensure predictions and references are in the correct format

        results = code_eval_metric.compute(
            predictions=responses,
            references=canonical_solution,  # Correct format: list of solutions repeated
            k=k_values
        )
        pass_at_k_dict = results[0]  # <--- This is the key change

        pass_at_k = {}
        for k in k_values:
            pass_at_k[f"pass@{k}"] = pass_at_k_dict.get(f"pass@{k}") # Use .get to handle missing keys
        return pass_at_k
    except Exception as e:
        print(f"Error during evaluation: {e}")
        raise e

def extract_python_code(text: str) -> str:
    """
    Extracts Python code enclosed in triple backticks (```python ... ```) from a string.

    Args:
        text: The input string.

    Returns:
        The extracted Python code as a string, or the original text if no such block is found.
    """
    match = re.search(r"```python\n(.*?)\n```", text, re.DOTALL)  # Use DOTALL to match across lines
    if match:
        return match.group(1).strip()  # .strip() to remove leading/trailing whitespace
    return text



In [ ]:

# Store evaluation results
base_eval_results = []
fine_tuned_eval_results = []

for eval_item, non_fine_tuned, fine_tuned in zip(dataset, base_results, fine_tuned_results):
    prompt = non_fine_tuned['prompt']
    canonical = extract_python_code(eval_item['response'])

    base_pass_at_k = evaluate_responses([[non_fine_tuned["code"]]], [canonical])
    fine_tuned_pass_at_k = evaluate_responses([[fine_tuned["code"]]], [canonical])
    # Store results
    base_eval_results.append({"prompt": prompt, "code": non_fine_tuned["code"], **base_pass_at_k})
    fine_tuned_eval_results.append({"prompt": prompt, "code": fine_tuned["code"], **fine_tuned_pass_at_k})

In [ ]:
import pandas as pd
from tabulate import tabulate

# Convert the evaluation results to DataFrames
base_df = pd.DataFrame(base_eval_results)
fine_tuned_df = pd.DataFrame(fine_tuned_eval_results)

# Filter out rows where pass@1 is not zero for both DataFrames
base_df_filtered = base_df[base_df["pass@1"] != 0]  # Or base_df["pass@1"] != 0.0 if it's a float
fine_tuned_df_filtered = fine_tuned_df[fine_tuned_df["pass@1"] != 0] # Or fine_tuned_df["pass@1"] != 0.0


# Customize columns to display
columns_to_display = ["prompt", "code", "pass@1"] # pass@k to pass@1

# Display Base Model Results using Tabulate (filtered)
print("Base Model Results (pass@1 != 0):")
if not base_df_filtered.empty:  # Check if the filtered DataFrame is empty
    print(tabulate(base_df_filtered[columns_to_display], headers='keys', tablefmt='grid', showindex=False))
else:
    print("No results to display (all pass@1 values are 0).")

# Display Fine-Tuned Model Results using Tabulate (filtered)
print("\nFine-Tuned Model Results (pass@1 != 0):")
if not fine_tuned_df_filtered.empty:
    print(tabulate(fine_tuned_df_filtered[columns_to_display], headers='keys', tablefmt='grid', showindex=False))
else:
    print("No results to display (all pass@1 values are 0).")


# Display lengths of the filtered DataFrames
print(f"\nLength of Base Model Results (pass@1 != 0): {len(base_df_filtered)}")
print(f"Length of Fine-Tuned Model Results (pass@1 != 0): {len(fine_tuned_df_filtered)}")


#If you still want to display the pass@k for k different from 1, you will need to filter each column individually

# Filter out rows where pass@k is not zero for both DataFrames
base_df_filtered_k = base_df[
    (base_df["pass@1"] != 0)
    ]
fine_tuned_df_filtered_k = fine_tuned_df[
    (fine_tuned_df["pass@1"] != 0)
    ]


# Customize columns to display
columns_to_display_k = ["prompt", "code", "pass@1"]

# Display Base Model Results using Tabulate (filtered)
print("\nBase Model Results (pass@1 != 0):")
if not base_df_filtered_k.empty:  # Check if the filtered DataFrame is empty
    print(tabulate(base_df_filtered_k[columns_to_display_k], headers='keys', tablefmt='grid', showindex=False))
else:
    print("No results to display (all pass@1 values are 0).")

# Display Fine-Tuned Model Results using Tabulate (filtered)
print("\nFine-Tuned Model Results (pass@1 != 0):")
if not fine_tuned_df_filtered_k.empty:
    print(tabulate(fine_tuned_df_filtered_k[columns_to_display_k], headers='keys', tablefmt='grid', showindex=False))
else:
    print("No results to display (all pass@1 values are 0).")


# Display lengths of the filtered DataFrames
print(f"\nLength of Base Model Results (pass@1 != 0): {len(base_df_filtered_k)}")
print(f"Length of Fine-Tuned Model Results (pass@10 != 0): {len(fine_tuned_df_filtered_k)}")

Base Model Results (pass@1 != 0):
No results to display (all pass@1 values are 0).

Fine-Tuned Model Results (pass@1 != 0):
+------------------------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------------------------------+----------+
| prompt                                                                                                           | code                                                                                                             |   pass@1 |
+==================================================================================================================+==================================================================================================================+==========+
| def flip_case(string: str) -> str:                                                                               | def flip_case(string: str) -> 

**Complex checking** holy shit me vs God he loses

In [ ]:
import ast
import radon.complexity
import io
import subprocess  # For running the cc command
import json
from pylint import lint
from pylint.reporters.json_reporter import JSONReporter
import tempfile  # For creating a temporary file
import zss
import math


class WritableObject(object):
    "dummy output stream for pylint"
    def __init__(self):
        self.content = []
    def write(self, st):
        "dummy write"
        self.content.append(st)
    def read(self):
        "dummy read"
        return self.content

def ast_to_tree(node):
    """Convert an AST node to a zss-compatible tree."""
    if isinstance(node, ast.AST):
        children = [ast_to_tree(child) for child in ast.iter_child_nodes(node)]
        return zss.Node(type(node).__name__, children)
    else:
        return zss.Node(str(node))

def sigmoid(x, k=1, x0=0):
    """Sigmoid function to transform distance into similarity."""
    return 1 / (1 + math.exp(-k * (x - x0)))

def normalized_ast_distance(code1, code2):
    """Calculate normalized tree edit distance."""
    ast1 = ast.parse(code1)
    ast2 = ast.parse(code2)
    tree1 = ast_to_tree(ast1)
    tree2 = ast_to_tree(ast2)
    distance = zss.simple_distance(tree1, tree2)
    max_distance = len(list(ast.walk(ast1))) + len(list(ast.walk(ast2)))
    return distance / max_distance

def ast_similarity(code1, code2, k=10, x0=0.5):
    """Calculate similarity between two ASTs using sigmoid."""
    distance = normalized_ast_distance(code1, code2)
    similarity = sigmoid(distance, k=k, x0=x0)
    return similarity

def evaluate_code(code, canonical_solution_ast=None):
    metrics = {}

    # 1. Syntactic Correctness & Static Analysis
    try:
        ast.parse(code)
        metrics["syntactic_correct"] = True
    except SyntaxError as e:
        metrics["syntactic_correct"] = False
        metrics["syntax_error"] = str(e)

    with tempfile.NamedTemporaryFile(mode='w', suffix='.py', delete=False) as temp_file:
        temp_filename = temp_file.name
        temp_file.write(code)
    pylint_output = WritableObject()
    json_reporter = JSONReporter(pylint_output)

    # Pylint options
    pylint_opts = [
        "--disable=missing-module-docstring,C0301",  # Suppress some messages
        "--rcfile=/dev/null",  # Disable rc file
        temp_filename,  # The temporary filename
    ]

    # Run pylint
    lint.Run(pylint_opts, reporter=json_reporter, exit=False)


    pylint_json_output = pylint_output.read()
    for a in pylint_json_output:
      print(a)
    print("pylint_json_output"+str(pylint_json_output))
    try:
      pylint_results = json.loads(''.join(pylint_json_output))
      metrics["pylint_errors"] = len([m for m in pylint_results if m.get("type") == "error"])
      metrics["pylint_warnings"] = len([m for m in pylint_results if m.get("type") == "warning"])
    except json.JSONDecodeError:
        metrics["pylint"] = "Failed to parse pylint output."


    os.remove(temp_filename) # Clean up the temporary file


    # 2. Code Complexity
    try:
        process = subprocess.run(['radon', 'cc', '-j', '-'],  # -j for JSON output, - for stdin
                               capture_output=True, input=code, text=True, check=True)
        radon_output = json.loads(process.stdout)
        if type(radon_output) is dict:
            metrics["avg_complexity"] = 0
        else:
          metrics["avg_complexity"] = sum(int(item['complexity']) for item in radon_output["-"]) / len(radon_output["-"]) if radon_output else 0
    except subprocess.CalledProcessError as e:
        metrics["avg_complexity"] = 0
        print(f"Radon cc command error: {e}")
    except json.JSONDecodeError as e:
        metrics["avg_complexity"] = 0
        print(f"Radon JSON decode error: {e}")
    except FileNotFoundError as e:
        metrics["avg_complexity"] = 0
        print(f"Radon cc not found: {e}. Is Radon installed?")

    # 3. Code Quality (Basic)
    metrics["code_length"] = len(code)

    # 4. Algorithmic Similarity (AST-based, if canonical solution is provided)
    if canonical_solution_ast:
        try:
            metrics["ast_similarity"] = 1.0- float(ast_similarity(code, canonical_solution_ast)) # Use custom similarity function

        except Exception as e:
            metrics["ast_similarity"] = 0
            print(f"AST comparison error: {e}")
    else:
        metrics["ast_similarity"] = 0

    return metrics

# Example usage

#code1 = """
#def foo(x):
#    return x + 1
#"""

#code2 = """
#def bar(y):
 #   return y - 1
#"""

#a= evaluate_code(code1, code2)
#print(a)


In [ ]:
# Store evaluation results
base_eval_results = []
fine_tuned_eval_results = []

for eval_item, non_fine_tuned, fine_tuned in zip(dataset, base_results, fine_tuned_results):
    prompt = non_fine_tuned['prompt']
    canonical = extract_python_code(eval_item['response'])

    base_pass = evaluate_code(non_fine_tuned["code"], canonical)
    fine_tuned_pass = evaluate_code(fine_tuned["code"], canonical)
    # Store results
    base_eval_results.append({"prompt": prompt, "code": non_fine_tuned["code"], **base_pass})
    fine_tuned_eval_results.append({"prompt": prompt, "code": fine_tuned["code"], **fine_tuned_pass})

Output streaming troncato alle ultime 5000 righe.
        "message-id": "E0001"
    }
]


pylint_json_output['[\n    {\n        "type": "error",\n        "module": "tmp06i0a9oi",\n        "obj": "",\n        "line": 27,\n        "column": 22,\n        "endLine": null,\n        "endColumn": null,\n        "path": "/tmp/tmp06i0a9oi.py",\n        "symbol": "syntax-error",\n        "message": "Parsing failed: \'invalid decimal literal (tmp06i0a9oi, line 27)\'",\n        "message-id": "E0001"\n    }\n]', '\n']
AST comparison error: invalid decimal literal (<unknown>, line 27)
[
    {
        "type": "error",
        "module": "tmpc0s6htm6",
        "obj": "",
        "line": 1,
        "column": 10,
        "endLine": null,
        "endColumn": null,
        "path": "/tmp/tmpc0s6htm6.py",
        "symbol": "syntax-error",
        "message": "Parsing failed: 'invalid syntax (tmpc0s6htm6, line 1)'",
        "message-id": "E0001"
    }
]


pylint_json_output['[\n    {\n        "type": "error",

In [ ]:
import pandas as pd
from tabulate import tabulate

# Convert the evaluation results to DataFrames
base_df = pd.DataFrame(base_eval_results)
fine_tuned_df = pd.DataFrame(fine_tuned_eval_results)

# Filter rows based on the specified conditions
base_df_filtered = base_df[
    (base_df["syntactic_correct"] == True) |
    (base_df["pylint_errors"] == 0) |
    (base_df["ast_similarity"] > 0.8)
]

fine_tuned_df_filtered = fine_tuned_df[
    (fine_tuned_df["syntactic_correct"] == True) |
    (fine_tuned_df["pylint_errors"] == 0) |
    (fine_tuned_df["ast_similarity"] > 0.8)
]

# Customize columns to display (including 'prompt' and 'code')
columns_to_display = ["prompt", "code", "syntactic_correct", "pylint_errors", "pylint_warnings", "avg_complexity", "code_length", "ast_similarity"]

# Display Base Model Results using Tabulate (filtered)
print("Base Model Results (filtered):")
if not base_df_filtered.empty:  # Check if the filtered DataFrame is empty
    print(tabulate(base_df_filtered[columns_to_display], headers='keys', tablefmt='grid', showindex=False))
else:
    print("No results to display (no rows match the criteria).")

# Display Fine-Tuned Model Results using Tabulate (filtered)
print("\nFine-Tuned Model Results (filtered):")
if not fine_tuned_df_filtered.empty:
    print(tabulate(fine_tuned_df_filtered[columns_to_display], headers='keys', tablefmt='grid', showindex=False))
else:
    print("No results to display (no rows match the criteria).")

# Display lengths of the filtered DataFrames
print(f"\nLength of Base Model Results (filtered): {len(base_df_filtered)}")
print(f"Length of Fine-Tuned Model Results (filtered): {len(fine_tuned_df_filtered)}")

Base Model Results (filtered):
No results to display (no rows match the criteria).

Fine-Tuned Model Results (filtered):
+------------------------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------------------------------+---------------------+-----------------+-------------------+------------------+---------------+------------------+
| prompt                                                                                                           | code                                                                                                             | syntactic_correct   |   pylint_errors |   pylint_warnings |   avg_complexity |   code_length |   ast_similarity |
+==================================================================================================================+=======================================================================